# Hyperparameter Sweep Job

This notebook demonstrates how to configure a Sweep Job using Random Sampling to tune the `learning_rate` parameter.

In [ ]:
from azure.ai.ml import MLClient, command, Input
from azure.ai.ml.sweep import Choice, Uniform
from azure.ai.ml.constants import AssetTypes
from azure.identity import DefaultAzureCredential

# Workspace details
subscription_id = "<SUBSCRIPTION_ID>"
resource_group = "<RESOURCE_GROUP>"
workspace_name = "<WORKSPACE_NAME>"

# Connect to the workspace
ml_client = MLClient(
    DefaultAzureCredential(), subscription_id, resource_group, workspace_name
)

## Define the Base Command

First, we define the command job that acts as the trial. Note that `learning_rate` is not defined in `inputs` here because it will be swept over.

In [ ]:
# Configure the base command job
job = command(
    code="./src",
    command="python train_model.py --data ${{inputs.diabetes}} --learning_rate ${{inputs.learning_rate}}",
    inputs={
        "diabetes": Input(
            type=AssetTypes.URI_FILE, 
            path="azureml:diabetes-data:1"
        ),
        # learning_rate will be provided by the sweep job
    },
    environment="azureml:AzureML-sklearn-1.0-ubuntu20.04-py38-cpu@latest",
    compute="cpu-cluster",
    display_name="diabetes-sweep-job",
    experiment_name="diabetes-hyperparameter-tuning"
)

## Configure the Sweep

We use the `.sweep()` method on the command object.

*   `sampling_algorithm`: We use "random" for Random Sampling.
*   `primary_metric`: The metric to optimize (must be logged by the script).
*   `goal`: "Maximize" or "Minimize".
*   `search_space`: The dictionary defining the hyperparameter space.

In [ ]:
# Configure the sweep job
job_for_sweep = job.sweep(
    compute="cpu-cluster",
    sampling_algorithm="random",
    primary_metric="r2",
    goal="Maximize",
)

# Define the search space
job_for_sweep.search_space = {
    "learning_rate": Uniform(min_value=0.01, max_value=0.5)
}

# Set limits
job_for_sweep.set_limits(max_total_trials=10, max_concurrent_trials=2, timeout=600)

In [ ]:
# Submit the sweep job
returned_sweep_job = ml_client.jobs.create_or_update(job_for_sweep)
print(f"Sweep Job created with URI: {returned_sweep_job.studio_url}")